In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class LeNet:
  def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
    model.add(Conv2D(20, (5,5), padding='same', input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Conv2D(50, (5,5), padding='same'))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

In [0]:
import matplotlib
matplotlib.use("Agg")

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [0]:
# HYPERPARAMETERS
EPOCHS = 25
INIT_LR = 1e-3
BS=1

# DATA
data=[]
labels=[]

imagepaths = sorted(list(paths.list_images("drive/My Drive/Colab Notebooks/cat pictures")))
random.seed(69)
random.shuffle(imagepaths)

In [0]:
for imagepath in imagepaths:
  image=cv2.imread(imagepath)
  image=cv2.resize(image, (28,28))
  image=img_to_array(image)
  data.append(image)

  label = imagepath.split(os.path.sep)[-2]
  label = 1 if label == "tail" else 0
  labels.append(label)

#print(data)
#print(labels)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

(trainx, testx, trainy, testy) = train_test_split(data, labels, test_size=0.25, random_state=69)

print(trainy)

trainy = to_categorical(trainy, num_classes=2)
testy = to_categorical(testy, num_classes=2)

[0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]


In [0]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [0]:
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [0]:
H = model.fit_generator(aug.flow(trainx, trainy, batch_size=BS), validation_data=(testx, testy), steps_per_epoch=len(trainx) // BS, epochs=EPOCHS, verbose=1)

model.save("image_model")

Epoch 1/25





7/7 [==============================] - 1s 211ms/step - loss: 3.7711 - acc: 0.4286 - val_loss: 0.0904 - val_acc: 1.0000
Epoch 2/25
7/7 [==============================] - 0s 25ms/step - loss: 0.9433 - acc: 0.5714 - val_loss: 0.8016 - val_acc: 0.0000e+00
Epoch 3/25
7/7 [==============================] - 0s 33ms/step - loss: 0.6998 - acc: 0.4286 - val_loss: 0.7447 - val_acc: 0.0000e+00
Epoch 4/25
7/7 [==============================] - 0s 29ms/step - loss: 0.6949 - acc: 0.4286 - val_loss: 0.7215 - val_acc: 0.0000e+00
Epoch 5/25
7/7 [==============================] - 0s 28ms/step - loss: 0.6960 - acc: 0.4286 - val_loss: 0.7010 - val_acc: 0.0000e+00
Epoch 6/25
7/7 [==============================] - 0s 27ms/step - loss: 0.6920 - acc: 0.7143 - val_loss: 0.6935 - val_acc: 0.3333
Epoch 7/25
7/7 [==============================] - 0s 31ms/step - loss: 0.7008 - acc: 0.4286 - val_loss: 0.6682 - val_acc: 1.0000
Epoch 8/25
7/7 [==============================] - 0s 25ms/step - loss: 0.69

In [0]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Tail/No Tail")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/content/drive/My Drive/image plots")

In [0]:
# Code for image prediction begins here

from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2

In [0]:
IMG = "benjinotail"
IMGPATH = "/content/drive/My Drive/Colab Notebooks/cat pictures/test/"+IMG+".png"
image = cv2.imread(IMGPATH)
orig = image.copy()

image = cv2.resize(image, (28, 28))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [0]:
(noTail, tail) = model.predict(image)[0]

In [0]:
label = "tail" if tail > noTail else "no tail"
prob = tail if tail > noTail else noTail
label = "{}: {:.2f}%".format(label, prob*100)

output = imutils.resize(orig, width=400)
cv2.putText(output, label, (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.imwrite("/content/drive/My Drive/Colab Notebooks/cat pictures/test/"+IMG+"prediction.png", output)

True